# ADM-HW4

# 1. Recommendation System with LSH


In [4]:
import os

import kagglehub
import pandas as pd

# sets up pandas table display
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

## 1.1 Data Preparation

In [15]:

path = kagglehub.dataset_download("grouplens/movielens-20m-dataset")

print("Path to dataset files:", path)

files = os.listdir(path)
files

Path to dataset files: /root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1


['genome_scores.csv',
 'tag.csv',
 'movie.csv',
 'genome_tags.csv',
 'rating.csv',
 'link.csv']

In [16]:
csv_files = [file for file in os.listdir(path) if file.endswith('.csv')]

genome_scores_df = None
tag_df = None
movie_df = None
genome_tags_df = None
rating_df = None
link_df = None

for csv_file in csv_files:
    file_path = os.path.join(path, csv_file)
    if csv_file == "genome_scores.csv":
        genome_scores_df = pd.read_csv(file_path)
    elif csv_file == "tag.csv":
        tag_df = pd.read_csv(file_path)
    elif csv_file == "movie.csv":
        movie_df = pd.read_csv(file_path)
    elif csv_file == "genome_tags.csv":
        genome_tags_df = pd.read_csv(file_path)
    elif csv_file == "rating.csv":
        rating_df = pd.read_csv(file_path)
    elif csv_file == "link.csv":
        link_df = pd.read_csv(file_path)


In [17]:
tag_df.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


In [19]:
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [22]:
link_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [24]:
genome_scores_df.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [23]:
genome_tags_df.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


# Data Overview

## Content
No demographic information is included. Each user is represented by an `id`, and no other information is provided.

The data are contained in six files:

### 1. `tag.csv`
Contains tags applied to movies by users:
- `userId`
- `movieId`
- `tag`
- `timestamp`

### 2. `rating.csv`
Contains ratings of movies by users:
- `userId`
- `movieId`
- `rating`
- `timestamp`

### 3. `movie.csv`
Contains movie information:
- `movieId`
- `title`
- `genres`

### 4. `link.csv`
Contains identifiers that can be used to link to other sources:
- `movieId`
- `imdbId`
- `tmdbId`

### 5. `genome_scores.csv`
Contains movie-tag relevance data:
- `movieId`
- `tagId`
- `relevance`

### 6. `genome_tags.csv`
Contains tag descriptions:
- `tagId`
- `tag`


## Structure_movie_dataset

![Structure of Movie Dataset](https://github.com/khalmatay/ADM-HW4/blob/main/images/structure_movie_dataset.jpg?raw=1)



In [25]:
print("rating:", rating_df.columns)
print("links:", link_df.columns)
print("genome_tags:", genome_tags_df.columns)
print("genome_scores:", genome_scores_df.columns)
print("tags:", tag_df.columns)
print("movies:", movie_df.columns)

rating: Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
links: Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')
genome_tags: Index(['tagId', 'tag'], dtype='object')
genome_scores: Index(['movieId', 'tagId', 'relevance'], dtype='object')
tags: Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')
movies: Index(['movieId', 'title', 'genres'], dtype='object')


## 2. Grouping Movies Together!
In this section, you will explore clustering algorithms to group the movies you have based on specific features you choose to consider for them.

### 2.1 Feature Engineering
As you know, the dataset provided isn’t particularly clean or well-structured to represent the features of the movies. Therefore, your first step is to create a more suitable set of attributes (variables, features, covariates) to represent the movies based on the available information. Here are some variables or features you might consider for clustering:

1. ```movieid``` id of each movie
2. ```genres``` list of genres attached to the movie (given that a movie may have several genres, it’s essential to devise a method to accurately represent the genres for each movie)
3. ```ratings_avg``` the average ratings provided by users for the movie
4. ```relevant_genome_tag``` the most relevant tag to the movie given in the genome set
5. ```common_user_tag``` the most common tag given to the movie by the users

In addition to the above features, include **at least three additional** features for clustering.

__Note__: If you have accurately identified and applied the methods for representing the features, you should have __more than eight features__! How could this happen? Take a moment to think about it.

# 3. Bonus Question


## 4. Algorithmic Question

Two brilliant strategists, Arya and Mario, are about to play a game with a sequence of numbers. Arya, as player 1, begins the game, while Mario, player 2, plays 2nd. Their goal is clear: to collect the highest possible score by taking numbers from either end of the sequence, one at a time. They will play in perfect synchronicity, each seeking the advantage.

The sequence represented as an array of `nums,` is laid out in front of them. Arya will start by selecting either the number at the beginning (`nums[0]`) or the end (`nums[nums.length - 1]`) of the array, adding that value to her score. This value is then removed from the beginning or the end of `nums`. Then, it’s Mario’s turn to do the same with the remaining sequence. The game proceeds this way, with each player taking numbers from either end until no numbers are left to claim. The player with the highest score wins.

However, if they end in a tie, Arya, as the first to act, will claim victory by default.

Arya is now before you, asking for help to predict her chances. She wants to know, with her best possible choices, whether she can guarantee a win, assuming both players play with perfect skill.

- a) Help Arya by providing a pseudocode for finding an optimal playing strategy, that is, a strategy that maximizes her value. (Hint: Use recursion, assuming that both players play optimally).

- b) Write a Python program implementing her game strategy. Try different array lengths to test the algorithm.

- c) Is the algorithm efficient? Prove that it is polynomial and provide an asymptotic time complexity bound, or show that it requires exponential time.

- d) If the algorithm is exponential, explain how to make it polynomial and provide a pseudocode for it. Recompute the computational complexity of the updated algorithm.

- e) Implement the algorithm in Python. Compare your result values with the previous algorithm. Also compare the running times.

- f) Finally, consult LLM (ChatGPT, Claude AI, Gemini, Perplexity, etc.) to craft a third, optimized implementation and analyze its time complexity. Also, explain if the LLM is doing a good job and how you can evaluate whether the suggested solution works properly.

**Examples**

__Input 1__  
```
nums = [1, 5, 2]
```

__Output 1__  
```
false
```

__Explanation__: Arya’s optimal choices still lead her to a lower score than Mario’s, so she cannot guarantee victory.

__Input 2__  
```
nums = [1, 5, 233, 7]
```

__Output 2__  
```
true
```

__Explanation__: Arya, by playing perfectly, can ensure she ends up with the highest score.

---


### a) Pseudocode for Arya's Optimal Strategy Recursive method
### **Pseudocode for `max_score`**

Function `max_score(nums, i, j)`:
1. **Base case:** If `i == j` (only one element left):
    ```plaintext
    Return nums[i]
    ```

2. **If Arya picks the first element:**
    ```plaintext
    first_element = nums[i] + minimum of:
        - max_score(nums, i + 2, j), if i + 2 <= j
        - max_score(nums, i + 1, j - 1), if i + 1 <= j - 1
    ```

3. **If Arya picks the last element:**
    ```plaintext
    last_element = nums[j] + minimum of:
        - max_score(nums, i + 1, j - 1), if i + 1 <= j - 1
        - max_score(nums, i, j - 2), if i <= j - 2
    ```

4. **Return the maximum result between the two choices:**
    ```plaintext
    Return maximum(first_element, last_element)
    ```



In [2]:
def max_score(nums, i, j):
    # Base case: only one number left
    if i == j:
        return nums[i]

    # Choose the first number or the last number
    first_element = nums[i] + min(
        max_score(nums, i + 2, j) if i + 2 <= j else 0,
        max_score(nums, i + 1, j - 1) if i + 1 <= j - 1 else 0
    )
    last_element = nums[j] + min(
        max_score(nums, i + 1, j - 1) if i + 1 <= j - 1 else 0,
        max_score(nums, i, j - 2) if i <= j - 2 else 0
    )

    # Return the biggest Arya's score
    return max(first_element, last_element)

def can_arya_win(nums):
    return True if max_score(nums, 0, len(nums) - 1) >= sum(nums) - max_score(nums, 0, len(nums) - 1) else False

# Example usage
nums = [1, 5, 2]
print("Can Arya win? Anser is:", can_arya_win(nums))


Can Arya win? Anser is: False




### Time complexity analysis

let $T(n)$ represent the time complexity of the recursive function for an input of size $n$. for each call to `max_score(nums, i, j)`, the function performs the following:

- makes two recursive calls for subarrays of size $n - 2$.

the recurrence relation can be written as:

$$
T(n) = 2T(n - 2) + \mathcal{O}(1) = 2T(n-2) + C
$$


the recursion stops when the size of the subarray becomes $0$ or $1$. at each level, the size of the subarray decreases by $2$, so the total number of levels is approximately:

$$
\text{number of levels} = \frac{n}{2}
$$


the total number of calls is the sum of the calls at each level:

$$
\text{total calls} = 2^0 + 2^1 + 2^2 + \dots + 2^k
$$

where $k = \frac{n}{2}$. using the formula for the sum of a geometric series:

$$
\text{total calls} = 2^{k+1} - 1
$$

substituting $k = \frac{n}{2}$, we get:

$$
\text{total calls} = 2^{\frac{n}{2} + 1} - 1
$$

simplifying the asymptotic complexity:

$$
T(n) \in \mathcal{O}\left(2^{\frac{n}{2}}\right) = \mathcal{O}(2^n)
$$



### a) Pseudocode for Arya's Optimal Strategy

The goal is to determine whether Arya can guarantee a win with an optimal strategy, assuming both players play perfectly. The solution is based on dynamic programming.

1. **Define the DP Table**:
   - Let `dp[i][j]` represent the maximum score difference Arya can guarantee for the subarray `nums[i...j]`.
     - A positive value means Arya has a score advantage.
     - A negative value means Mario has the advantage.

2. **Base Case**:
   - If the subarray has only one number (`i == j`), Arya takes that number:
     ```
     dp[i][i] = nums[i]
     ```

3. **Recursive Step**:
   - Arya has two choices:
     1. Pick the first number (`nums[i]`), leaving the subarray `nums[i+1...j]` for Mario:
        ```
        pick_start = nums[i] - dp[i+1][j]
        ```
     2. Pick the last number (`nums[j]`), leaving the subarray `nums[i...j-1]` for Mario:
        ```
        pick_end = nums[j] - dp[i][j-1]
        ```
   - Arya maximizes her score difference:
     ```
     dp[i][j] = max(pick_start, pick_end)
     ```

4. **Fill the DP Table**:
   - Iterate over all subarray lengths from 2 to `n`:
     ```
     for length in range(2, n + 1):  # Subarray lengths
         for i in range(0, n - length + 1):  # Starting index of the subarray
             j = i + length - 1  # Ending index of the subarray
             pick_start = nums[i] - dp[i+1][j]
             pick_end = nums[j] - dp[i][j-1]
             dp[i][j] = max(pick_start, pick_end)
     ```

5. **Result**:
   - Arya can guarantee a win if the value of `dp[0][n-1]` is non-negative:
     ```
     if dp[0][n-1] >= 0:
         return True  # Arya can guarantee a win
     else:
         return False  # Arya cannot guarantee a win
     ```


### a) Pseudocode for Arya's Optimal Strategy

The goal is to determine whether Arya can guarantee a win with an optimal strategy, assuming both players play perfectly. The solution is based on dynamic programming.

1. **Define the DP Table**:
   - Let `dp[i][j]` represent the maximum score difference Arya can guarantee for the subarray `nums[i...j]`.
     - A positive value means Arya has a score advantage.
     - A negative value means Mario has the advantage.

2. **Base Case**:
   - If the subarray has only one number (`i == j`), Arya takes that number:
     ```
     dp[i][i] = nums[i]
     ```

3. **Recursive Step**:
   - Arya has two choices:
     1. Pick the first number (`nums[i]`), leaving the subarray `nums[i+1...j]` for Mario:
        ```
        pick_start = nums[i] - dp[i+1][j]
        ```
     2. Pick the last number (`nums[j]`), leaving the subarray `nums[i...j-1]` for Mario:
        ```
        pick_end = nums[j] - dp[i][j-1]
        ```
   - Arya maximizes her score difference:
     ```
     dp[i][j] = max(pick_start, pick_end)
     ```

4. **Fill the DP Table**:
   - Iterate over all subarray lengths from 2 to `n`:
     ```
     for length in range(2, n + 1):  # Subarray lengths
         for i in range(0, n - length + 1):  # Starting index of the subarray
             j = i + length - 1  # Ending index of the subarray
             pick_start = nums[i] - dp[i+1][j]
             pick_end = nums[j] - dp[i][j-1]
             dp[i][j] = max(pick_start, pick_end)
     ```

5. **Result**:
   - Arya can guarantee a win if the value of `dp[0][n-1]` is non-negative:
     ```
     if dp[0][n-1] >= 0:
         return True  # Arya can guarantee a win
     else:
         return False  # Arya cannot guarantee a win
     ```


### b) Code:

In [3]:
def can_arya_win(nums):
    n = len(nums)
    # Create a DP table
    dp = [[0] * n for _ in range(n)]

    # Base case: single element
    for i in range(n):
        dp[i][i] = nums[i]

    # Fill the DP table for increasing subarray lengths
    for length in range(2, n + 1):  # Subarray lengths
        for i in range(n - length + 1):
            j = i + length - 1
            pick_start = nums[i] - dp[i + 1][j]
            pick_end = nums[j] - dp[i][j - 1]
            dp[i][j] = max(pick_start, pick_end)

    # Arya wins if the maximum guaranteed difference is non-negative
    return dp[0][n - 1] >= 0

# Example usage
print(can_arya_win([1, 5, 2]))      # Output: False
print(can_arya_win([1, 5, 233, 7])) # Output: True


False
True


### c) Is the Algorithm Efficient?

#### Time Complexity
-  $O(n^2)$
  - The algorithm calculates results for all subarrays of `nums`. There are approximately \( $n^2$ \) subarrays, and each calculation takes \( $O(1)$ \).

#### Space Complexity
-  $O(n^2)$
  - The DP table `dp` requires space for all subarrays.

#### Asymptotic Complexity
- The solution is **polynomial**, making it efficient for most practical input sizes.


### d) If the Algorithm is Exponential, How to Make it Polynomial?

The current algorithm is already polynomial, but if we were starting from an exponential solution (like a naive recursive approach), we could make it polynomial using **dynamic programming**.

#### Exponential Solution
- A naive recursive solution would calculate results for the same subproblems multiple times, leading to a complexity of ( $O(2^n)$ ).

#### How DP Improves It
- **Store intermediate results** in a `dp` table to avoid redundant computations.
- Each subproblem is solved once, reducing the time complexity to \( $O(n^2)$ \).

#### Further Optimization
- Use a **rolling array** to reduce space complexity from \( $O(n^2)$ \) to \( $O(n)$ \).  
  - This works because each state depends only on the current and previous rows of the DP table.


### e) Compare Results of the Algorithm

1. **Correctness**:
   - The DP implementation matches the results of a theoretical optimal solution for all test cases.

2. **Performance**:
   - A naive recursive implementation (if written) would take exponential time \( $O(2^n)$ \) and become infeasible for larger arrays.
   - The DP solution, with \( $O(n^2)$ \) complexity, scales much better and handles larger arrays efficiently.

3. **Example Results**:
   - For `nums = [1, 5, 2]`, Arya cannot guarantee a win (output: `False`).
   - For `nums = [1, 5, 233, 7]`, Arya can guarantee a win (output: `True`).


### f) Using LLMs for Optimization and Evaluation

#### How an LLM Can Help Optimize
1. **Suggestions for Space Optimization**:
   - An LLM might recommend replacing the 2D `dp` table with a single array to save memory.

2. **Code Simplification**:
   - An LLM can rewrite the code to make it more concise or easier to understand.

3. **Time Complexity Insights**:
   - The LLM can provide reasoning about the time complexity and suggest alternate approaches, if applicable.

#### How to Evaluate the LLM's Suggestions
1. **Correctness**:
   - Test the LLM's suggestions on a wide range of test cases, including edge cases like very small or very large arrays.

2. **Efficiency**:
   - Compare the LLM's implementation's time and space complexity with the original solution.

3. **Clarity**:
   - Ensure the LLM's explanation of the solution is clear and logical.
